In [1]:
import os
import sys 

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config("spark.sql.warehouse.dir", "C:/Users/JSM")\
    .getOrCreate()

In [5]:
#파일 읽기(Python)
import os

try:
    fp=os.path.join(os.environ["SPARK_HOME"],\
        'data', 'mllib', 'sample_svm_data.txt')
except:
    print("error occured!")
    
f=open(fp, 'r')
lines = f.readlines() #함수를 통해 파일로부터 모든 데이터를 읽어온다.(한 줄씩)
f.close()

print lines[0]

1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0



In [8]:
#RDD 생성(Spark)
rdd = spark.sparkContext.textFile(fp)\
    .map(lambda line: [float(x) for x in line.split(' ')])

rdd.take(2)[0]

[1.0,
 0.0,
 2.52078447201548,
 0.0,
 0.0,
 0.0,
 2.004684436494304,
 2.000347299268466,
 0.0,
 2.228387042742021,
 2.228387042742023,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [9]:
#LabeledPoint 생성
from pyspark.mllib.regression import LabeledPoint

trainRdd0 = rdd.map(lambda line: LabeledPoint(line[0], line[1:])) #1: 1~ 중요한 문법

In [11]:
trainRdd0.take(1)

[LabeledPoint(1.0, [0.0,2.52078447201548,0.0,0.0,0.0,2.004684436494304,2.000347299268466,0.0,2.228387042742021,2.228387042742023,0.0,0.0,0.0,0.0,0.0,0.0])]

In [12]:
#훈련데이터 함수 만들기
def createLP(line):
    p = [float(x) for x in line.split(' ')]
    return LabeledPoint(p[0], p[1:])

_rdd=spark.sparkContext.textFile(fp)
_trainRdd = _rdd.map(createLP) #lambda는 사용자 정의 함수, 함수를 넣을 때는 정의할 필요 없어요.

_trainRdd.take(1)


[LabeledPoint(1.0, [0.0,2.52078447201548,0.0,0.0,0.0,2.004684436494304,2.000347299268466,0.0,2.228387042742021,2.228387042742023,0.0,0.0,0.0,0.0,0.0,0.0])]